In [ ]:
## 사전 실행 코드
import polars as pl
import pycountry_convert as pc
def get_continent_name(nation_code: str) -> str:
	if nation_code != 'WW':
		continent_code = pc.country_alpha2_to_continent_code(nation_code)
	else: 
		continent_code = 'WW'
	continent_dict = {"NA": "North America","SA": "South America", "AS": "Asia", "AF": "Africa",
		"OC": "Oceania", "EU": "Europe", "AQ": "Antarctica", "WW": "Global"}
	return continent_dict[continent_code]

key_levels = pl.Enum(["C", "C#", "D", "Eb", "E", "F", "F#", "G", "G#", "A", "Bb", "B"])

df_spotify = (pl.read_csv("./universal_top_spotify_songs.csv", try_parse_dates = True, null_values = [""])
	.filter(pl.col('snapshot_date').dt.year() == 2024).sort('snapshot_date')
    .with_columns(pl.when(pl.col('country').is_null() == True)  ## country 열이 null이면
		.then(pl.lit('WW')) ## WW로 변경
		.otherwise(pl.col('country')).alias('country'))  ## 아니면 원래대로
	.drop_nulls()
    .with_columns(pl.col('key').cast(pl.String)
	.replace(["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11"],
		["C", "C#", "D", "Eb", "E", "F", "F#", "G", "G#", "A", "Bb", "B"]))
	.with_columns(pl.col('key').cast(key_levels)).sort('key')
    .with_columns(pl.col('artists').str.split(', '))  ## ,를 기준으로 문자열을 분리
    ## 리스트의 첫 번째 아이템을 가져와서 main_vocal로 저장
	.with_columns(pl.col('artists').list.get(0, null_on_oob = True).alias('main_vocal'), 
		pl.col('artists').list.tail(-1).alias('featuring')) ## 첫 번째 아이템을 제외한 나머지를 featuring으로 저장
	.with_columns(pl.when(pl.col('featuring').list.len() == 0) ## 리스트 길이가 0이면
		.then(None)  ## None으로 설정
		.otherwise(pl.col('featuring')).name.keep())
    .with_columns(pl.col('country')
        .map_elements(get_continent_name, return_dtype = pl.String).alias('continent')))

# 10장	스포티파이 음악 데이터로 인사이트 도출하기

[10장 결과확인: https://estndard.github.io/polars/10.html](https://estndard.github.io/polars/10.html)

## 10.1	스포티파이 숫자 데이터로 국가별 인기도 분포 분석하기

In [ ]:
pl.Config(set_tbl_cols = 10, set_tbl_rows = 25)
(df_spotify.select(pl.col([pl.Int64, pl.Float64])).describe()
	.transpose(include_header = True, header_name = 'columns',
		column_names = ["count", "null_count", "mean", "std", "min", "25%", "50%", "75%", "max"])[1:])

In [ ]:
import plotly.express as px
fig = px.histogram(df_spotify, x = 'daily_rank') 
fig.show()

In [ ]:
fig = px.histogram(df_spotify, x = 'daily_movement') 
fig.show()

In [ ]:
fig = px.histogram(df_spotify, x = 'weekly_movement') 
fig.show()

In [ ]:
fig = px.histogram(df_spotify, x = 'popularity') 
fig.show()

In [ ]:
fig = px.pie(df_spotify.group_by('is_explicit').len('count'), values = "count")
fig.update_traces(textinfo = "percent+label")
fig.show()

In [ ]:
pl.Config(set_tbl_cols = 10, set_tbl_rows = 25)
(df_spotify.select(pl.col([pl.String, pl.List(pl.String)])). describe()
	.transpose(include_header = True, header_name = 'columns', 
		column_names = ['count', 'null_count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'])[1:])

In [ ]:
df_spotify.select(pl.col([pl.String]).unique().len())

In [ ]:
fig = px.bar((df_spotify.group_by('country').agg(pl.len().alias('count'))
    .with_columns(pl.when(pl.col('country') == "KR").then(pl.lit("KR"))
    	.when(pl.col('country') == "WW").then(pl.lit("Global")).otherwise(pl.lit("Others")).alias('국가'))),
	x = 'country', y = 'count', color = '국가')
fig.update_xaxes(categoryorder = "total descending")
fig.show()

In [ ]:
df_spotify.group_by('country').len().sort('len', descending = True)

In [ ]:
fig = px.bar(df_spotify.group_by('continent').agg(pl.col('country').unique().len()), 
	x = 'continent', y = 'country', text = 'country')
fig.update_xaxes(categoryorder = "total descending")
fig.show()

In [ ]:
(df_spotify.group_by('continent').agg(pl.col('country').unique().len()).sort('country', descending = True))

In [ ]:
fig = px.histogram(df_spotify, x = 'snapshot_date')
fig.show()

In [ ]:
fig = px.box(df_spotify, x = 'continent', y = 'popularity')
fig.show()

In [ ]:
fig = px.box((df_spotify. filter(pl.col('country').is_in(["WW", "KR", "US", "BR", "GB", "AU", "NG"]))), 
	x = 'country', y = 'popularity')
fig.show()

In [ ]:
(df_spotify.filter(pl.col('country').is_in(["KR", "GB", "WW", "US"]))
	.group_by('country', 'name').agg(pl.all().sort_by('snapshot_date').first())
	.select(pl.col('country', 'name'), (pl.col('snapshot_date')-pl.col('album_release_date')).alias('duration'))
	.group_by('country').agg(pl.col('duration').mean().dt.total_days().alias('duration_mean'),
		pl.col('duration').median().dt.total_days().alias('duration_median'))
	.sort('duration_mean', descending = True))

In [ ]:
fig = px.box((df_spotify.filter(pl.col('country').is_in(["KR", "GB", "WW", "US"]))
	.group_by('country', 'name').agg(pl.all().sort_by('snapshot_date').first())
	.select(pl.col('country', 'name'), 	
		(pl.col('snapshot_date')-pl.col('album_release_date')).dt.total_days().alias('duration'))),
	x = 'country', y = 'duration')
fig.show()

In [ ]:
fig = px.box(df_spotify.filter(pl.col('country').is_in(["WW", "KR", "US", "BR", "GB", "AU", "NG"])), 
	x = 'country', y = 'popularity', color = 'is_explicit')
fig.show()

In [ ]:
(df_spotify.filter(pl.col('country').is_in(["WW", "KR", "US", "BR", "GB", "AU", "NG"]))
	.group_by('country').agg(((pl.col('is_explicit')
	.filter(pl.col('is_explicit') == True).len()) / (pl.col('is_explicit').len())*100).round(2).alias('Explicit_True(%)'),
		((pl.col('is_explicit').filter(pl.col('is_explicit') == False).len()) / (pl.col('is_explicit').len())*100).round(2).alias('Explicit_False(%)'))
	.sort('Explicit_True(%)', descending = True))

## 10.2	수치형 변수 간 상관관계와 회귀 분석하기

In [ ]:
df_spotify.select(pl.col(pl.Int64), pl.col(pl.Float64)).corr()

In [ ]:
fig = px.imshow((df_spotify.select(pl.col(pl.Int64), pl.col(pl.Float64)).corr().with_columns(pl.all().round(1))),
	y = df_spotify.select(pl.col(pl.Int64), pl.col(pl.Float64)).columns,
	text_auto = True, aspect = "auto", color_continuous_scale = "RdBu_r")
fig.show()

In [ ]:
(df_spotify.select(pl.col(pl.Int64), pl.col(pl.Float64)).corr()
	.with_columns(index = pl.lit(pl.Series(df_spotify.select(pl.col(pl.Int64), pl.col(pl.Float64)).columns)))
	.unpivot(index = 'index')
	.filter(pl.col('index') != pl.col('variable'))
	.filter((pl.col('value') > 0.5) | (pl.col('value') < -0.5))
	.sort('value', descending = True))

In [ ]:
fig = px.scatter(df_spotify.sample(fraction = 0.1, seed = 123),
	x = 'loudness', y = 'energy', trendline = 'ols', trendline_color_override = "red", opacity = 0.1, 
	range_y = [0, 1])
fig.show()

In [ ]:
result = px.get_trendline_results(fig)
result.px_fit_results.iloc[0].summary()

In [ ]:
fig = px.scatter(df_spotify.sample(fraction = 0.1, seed = 123), 
	x = 'acousticness', y = 'energy', trendline_color_override = "red", trendline = "ols", opacity = 0.1)
fig.show()

In [ ]:
result = px.get_trendline_results(fig)
result.px_fit_results.iloc[0].summary()

## 10.3	스포티파이 데이터로 글로벌 인기도 파악하기

In [ ]:
expr_1 = pl.col('name').unique().len().over('main_vocal')

df_spotify_EDA1 = (
pl.concat([
    ## 글로벌 메인보컬 Top 10 산출
    (df_spotify.filter(pl.col('country') == 'WW').select(pl.col('main_vocal').alias('Global_Main_Vocal'),
	expr_1.alias('Global_Songs')).unique().sort('Global_Songs', descending = True).head(10)),
    ## 우리나라 메인보컬 Top 10 산출
    (df_spotify.filter(pl.col('country') == 'KR').select(pl.col('main_vocal').alias('KR_Main_Vocal'), 
		expr_1.alias('KR_Songs')).unique().sort('KR_Songs', descending = True).head(10)),
    ## 미국 메인보컬 Top 10 산출
    (df_spotify.filter(pl.col('country') == 'US').select(pl.col('main_vocal').alias('US_Main_Vocal'),
		expr_1.alias('US_Songs')).unique().sort('US_Songs', descending = True).head(10)),
    ## 영국 메인보컬 Top 10 산출
    (df_spotify.filter(pl.col('country') == 'GB').select(pl.col('main_vocal').alias('GB_Main_Vocal'),
		expr_1.alias('GB_Songs')).unique().sort('GB_Songs', descending = True).head(10))], 
	how = 'horizontal')
	.with_columns(pl.int_range(1, 11).alias('rank'))
	.select(pl.col('rank'), pl.all().exclude('rank')))
df_spotify_EDA1

In [ ]:
fig = px.bar(
    (df_spotify.filter(pl.col('country').is_in(["WW", "KR", "US", "GB"]))  ## 대상 국가 필터링
		.group_by('country', 'main_vocal')  ## 국가와 메인보컬로 그룹화
		.agg(pl.col('name').unique().len())  ## 노래 이름의 고윳값에 대한 개수 산출
		.sort(['country', 'name'], descending = True)  ## 결과를 국가와 노래 이름으로 정렬
		.group_by('country', maintain_order = True)  ## 결과를 국가명으로 다시 그룹화
		.head(10)),  ## 상위 10개만 선택
    x = 'main_vocal', y = 'name', text = 'name',  ## X축은 main_vocal, Y축과 막대 텍스트는 name으로 설정
    facet_row = 'country', facet_row_spacing = 0.07,  ## facet 설정
    labels = {"main_vocal": "메인보컬", "name": "노래수"})  ## 축 라벨 설정
fig.update_xaxes(matches = None, showticklabels = True)  ## X축 간의 매칭과 틱라벨을 제거 
fig.show()

In [ ]:
(df_spotify.filter(pl.col('country') == "WW",  ## 글로벌만 필터링
		pl.col('main_vocal') == "Taylor Swift")  ## 테일러 스위프트만 필터링
	.group_by(['main_vocal', 'name'])  ## 메인보컬과 노래명으로 그룹화
	.len('chart in days')  ## 전체 개수 산출
	.sort('chart in days', descending = True).head(10))

In [ ]:
(df_spotify.filter(pl.col('country') == "KR", pl.col('main_vocal') == "Jimin")
	.group_by(['main_vocal', 'name']).len('chart in days').sort('chart in days', descending = True).head(10))

In [ ]:
df_spotify_EDA2 = ( 
pl.concat([
	(df_spotify.filter(pl.col('country') == "WW")  ## 글로벌 차트만 필터링
		.select(pl.col('name').alias('Global_Song'),  ## 노래명 열 선택
			pl.col('main_vocal').alias('Global_Vocal'),  ## 메인보컬 열 선택
 			pl.col('name').len().over('name').alias('Global_Day'))  ## 노래명별 노래 수 산출
 
                                ## 고유 행만 산출하여 정렬 후 상위 10곡만 ㄹ출력
		.unique().sort('Global_Day', descending = True).head(10)),  
	(df_spotify.filter(pl.col('country') == "KR")  ## 한국 차트만 필터링
		.select(pl.col('name').alias('KR_Song'),
			pl.col('main_vocal').alias('KR_Vocal'),
			pl.col('name').len().over('name').alias('KR_Day'))
		.unique().sort('KR_Day', descending = True).head(10)),
	(df_spotify.filter(pl.col('country') == "US")  ## 미국 차트만 필터링
		.select(pl.col('name').alias('US_Song'),
			pl.col('main_vocal').alias('US_Vocal'),
			pl.col('name').len().over('name').alias('US_Day'))
		.unique().sort('US_Day', descending = True).head(10)),
	(df_spotify.filter(pl.col('country') == "GB")
		.select(pl.col('name').alias('GB_Song'),  ## 영국 차트만 핕터링
			pl.col('main_vocal').alias('GB_Vocal'),
			pl.col('name').len().over('name').alias('GB_Day'))
		.unique().sort('GB_Day', descending = True).head(10))],
	how = 'horizontal')
   .with_columns(pl.int_range(1, 11).alias('rank'))  ## 순위 열 생성
   .select(pl.col('rank'), pl.all().exclude('rank'))  ## 순위 열을 앞으로 재배치
)

df_spotify_EDA2

In [ ]:
(df_spotify_EDA2.style
	.tab_header(title = "2024년 노래 Top 10")  ## 표 제목 설정
	.tab_stub(rowname_col = 'rank')  ## 스텁 설정
    ## 스패너 설정
	.tab_spanner("글로벌", ['Global_Song', 'Global_Vocal', 'Global_Day'])
	.tab_spanner("한국", ['KR_Song', 'KR_Vocal', 'KR_Day'])
	.tab_spanner("미국", ['US_Song', 'US_Vocal', 'US_Day'])
	.tab_spanner("영국", ['GB_Song', 'GB_Vocal', 'GB_Day'])
    ## 열 정렬 설정
	.cols_align(align = "center")
    ## 열 라벨 설정
	.cols_label(Global_Song = "노래", Global_Vocal = "메인보컬", Global_Day = "차트일수", 
		KR_Song = "노래", KR_Vocal = "메인보컬", KR_Day = "차트일수", 
		US_Song = "노래", US_Vocal = "메인보컬", US_Day = "차트일수", 
		GB_Song = "노래", GB_Vocal = "메인보컬", GB_Day = "차트일수"))

In [ ]:
## 국가별, 메인보컬별, 노래별 차트일 수 붙이기
(pl.concat([
	(df_spotify.filter(pl.col('country') == "WW", pl.col('daily_rank') == 1)
		.group_by('name')
		.agg(pl.col('main_vocal').first().alias('Global_Main_Vocal'), pl.len().alias('Global_Chart_Days'))
		.rename({"name": "Global_Song"}).sort('Global_Chart_Days', descending = True).head(10)),
    (df_spotify.filter(pl.col('country') == "KR", pl.col('daily_rank') == 1)
		.group_by('name')
		.agg(pl.col('main_vocal').first().alias('KR_Main_Vocal'), pl.len().alias('KR_Chart_Days'))
		.rename({"name": "KR_Song"}).sort('KR_Chart_Days', descending = True).head(10)),
    (df_spotify.filter(pl.col('country') == "US", pl.col('daily_rank') == 1)
		.group_by('name')
		.agg(pl.col('main_vocal').first().alias('US_Main_Vocal'), pl.len().alias('US_Chart_Days'))
		.rename({"name": "US_Song"}).sort('US_Chart_Days', descending = True).head(10)),
    (df_spotify.filter(pl.col('country') == "GB", pl.col('daily_rank') == 1)
		.group_by('name')
		.agg(pl.col('main_vocal').first().alias('GB_Main_Vocal'), pl.len().alias('GB_Chart_Days'))
		.rename({"name": "GB_Song"}).sort('GB_Chart_Days', descending = True).head(10))],
    how = 'horizontal')
  	.with_columns(pl.int_range(1, 11).alias('rank'))  ## 순위 열 만들기
	.select(pl.col('rank'), pl.all().exclude('rank')).style   ## 순위 열 순서 설정
	.tab_header(title = "2024년 차트 1위 노래 Top 10")   ## 표 제목 설정
	.tab_stub(rowname_col = 'rank')
 	## 스패너 설정
.tab_spanner("글로벌", ['Global_Song', 'Global_Main_Vocal', 'Global_Chart_Days'])
	.tab_spanner("한국", ['KR_Song', 'KR_Main_Vocal', 'KR_Chart_Days'])
	.tab_spanner("미국", ['US_Song', 'US_Main_Vocal', 'US_Chart_Days'])
	.tab_spanner("영국", ['GB_Song', 'GB_Main_Vocal', 'GB_Chart_Days'])
 	.cols_align(align = "center")  ## 열 제목 정렬
## 열 라벨 설정
	.cols_label(Global_Song = "노래", Global_Main_Vocal = "메인보컬", Global_Chart_Days = "차트일수", 
		KR_Song = "노래", KR_Main_Vocal = "메인보컬", KR_Chart_Days = "차트일수", 
		US_Song = "노래", US_Main_Vocal = "메인보컬", US_Chart_Days = "차트일수", 
		GB_Song = "노래", GB_Main_Vocal = "메인보컬", GB_Chart_Days = "차트일수"))

In [ ]:
(df_spotify.filter(pl.col('country').is_in(["WW", "KR", "US", "GB"]),  ## 국가 필터링
		pl.col('daily_rank') == 1)  ## 1위만 필터링
	.select(pl.col('country'), pl.col('name').unique().len().over('country'))  ## 국가별 1위곡 수 산출 
	.unique().sort('name', descending = True))

In [ ]:
fig = px.line((df_spotify.
     ## 우리나라 1위곡들만 필터링
	filter(pl.col('name').is_in(["Like Crazy", "Who", "Magnetic", "How Sweet", "Supernova", "Supernatural"]),
		pl.col('country') == 'KR')),
	x = 'snapshot_date', y = 'daily_rank', color = 'name', line_dash = 'name',
	labels = {"snapshot_date": "날짜", "daily_rank": "순위", "name": "노래"})
fig.update_yaxes(autorange = "reversed")
fig.show()

In [ ]:
(df_spotify.filter(pl.col('name') == "APT.", pl.col('daily_rank') == 1)
	.select(pl.col('continent'), pl.col('country').unique().len().over('continent').alias('NO.1'))
	.unique().sort('NO.1', descending = True))

In [ ]:
(df_spotify.filter(pl.col('name') == "APT.", pl.col('daily_rank') == 1,
		pl.col('continent') == "Global").select(pl.col('snapshot_date')))

In [ ]:
(df_spotify.filter(pl.col('country') == "KR", pl.col('name') == "APT.").select(pl.col('daily_rank').min()))

In [ ]:
fig = px.line((df_spotify.filter(pl.col('country').is_in(["WW", "KR", "US", "GB"]), pl.col('name') == "APT.")), 
		x = 'snapshot_date', y = 'daily_rank', color = 'country', line_dash = 'country')
fig.update_yaxes(autorange = "reversed")
fig.show()

In [ ]:
df_spotify_EDA4 = (
    df_spotify.filter(pl.col('name') == "APT.").select(pl.col('country'), pl.col('continent'),
		pl.col('country').map_elements(
			lambda x: pc.country_name_to_country_alpha3(pc.country_alpha2_to_country_name(x))
				if x != "WW" else "WW", return_dtype = pl.String).alias('nation'),
					(pl.col('country').map_elements(
						lambda x: pc.country_alpha2_to_country_name(x) if x != "WW" else "WW", 
							return_dtype = pl.String).alias('nation_name')),
		pl.col('popularity'), pl.col('daily_rank'), pl.col('name').len().over('country').alias('chart_days'))
			.group_by('nation').agg(pl.col('country').first(),
		pl.col('nation_name').first(), pl.col('continent').first(),
		pl.col('popularity').mean(), pl.col('daily_rank').mean(),
		pl.col('chart_days').first()))
df_spotify_EDA4.sort('daily_rank')

In [ ]:
(df_spotify.select(pl.col('country').unique()).join(df_spotify_EDA4, on = 'country', how = "anti"))

In [ ]:
fig = px.choropleth(df_spotify_EDA4, locations = 'nation', color = 'popularity', scope = "world",
	hover_name = 'nation_name', color_continuous_scale = "greens", width = 800, height = 600,
	title = "로제의 APT. 인기도")
fig.show()

In [ ]:
fig = px.choropleth(df_spotify_EDA4, locations = 'nation', color = 'daily_rank', scope = "world", 
	hover_name = 'nation_name', color_continuous_scale = "greens_r", width = 800, height = 600,
	title = '로제의 아파트 평균 순위')
fig.show()